# 🌟 AURA NEXUS v30 - Sistema de Enriquecimento de Leads

**Notebook completo para Google Colab com todas as correções aplicadas**

Este notebook contém:
- ✅ Montagem do Google Drive e cópia de arquivos
- ✅ Instalação de dependências
- ✅ Aplicação de todas as correções (patches)
- ✅ Funções auxiliares para execução
- ✅ Exemplos de uso e análise de resultados

## 📁 1. Montar Google Drive e Copiar Arquivos

**Execute esta célula PRIMEIRO** para acessar os arquivos do AURA NEXUS no seu Google Drive:

In [ ]:
# Instalação de dependências essenciais
print("🔧 Instalando dependências necessárias...\n")

# Dependências essenciais
!pip install -q validators
!pip install -q phonenumbers
!pip install -q nest-asyncio
!pip install -q aiohttp
!pip install -q tenacity
!pip install -q python-dotenv

# Web scraping
!pip install -q beautifulsoup4
!pip install -q lxml
!pip install -q html5lib
!pip install -q apify-client  # ADICIONADO: Cliente Apify

# Planilhas
!pip install -q openpyxl
!pip install -q xlsxwriter

# APIs
!pip install -q googlemaps
!pip install -q google-api-python-client

# LLMs
!pip install -q openai
!pip install -q anthropic
!pip install -q google-generativeai

print("\n✅ Todas as dependências instaladas!")

# Verificar instalações principais
print("\n📋 Verificando instalações principais:")
import pkg_resources

packages_to_check = [
    'validators',
    'phonenumbers',
    'nest_asyncio',
    'aiohttp',
    'beautifulsoup4',
    'openpyxl',
    'googlemaps',
    'apify-client',  # ADICIONADO
    'openai'
]

for package in packages_to_check:
    try:
        # Ajustar nome do pacote para verificação
        check_name = package.replace('-', '_') if package == 'apify-client' else package
        version = pkg_resources.get_distribution(check_name).version
        print(f"   ✅ {package} v{version}")
    except:
        print(f"   ❌ {package} - NÃO INSTALADO")

## 📦 2. Instalação de Dependências

Execute esta célula para instalar todas as dependências necessárias:

In [ ]:
# =====================================================
# CORREÇÕES DO SISTEMA AURA NEXUS v30 - VERSÃO FINAL
# =====================================================

import logging
from typing import Dict, Any
from datetime import datetime
import nest_asyncio
import os
import time

# Permitir async no Jupyter
nest_asyncio.apply()

# Importar módulos necessários
from aura_nexus_celula_11_v4 import AuraNexusOrchestratorV4
from aura_nexus_celula_10_v2 import LeadProcessor, get_processor_config
from aura_nexus_celula_00 import APIManager  # CORREÇÃO: Importar do módulo correto

print("📦 Aplicando correções no sistema...\n")

# =====================================================
# VERIFICAR CONFIGURAÇÃO DE APIs
# =====================================================

print("🔍 Verificando configuração de APIs...\n")

# Verificar variáveis de ambiente essenciais
api_keys = {
    'GOOGLE_MAPS_API_KEY': os.getenv('GOOGLE_MAPS_API_KEY'),
    'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY'),
    'GOOGLE_CSE_API_KEY': os.getenv('GOOGLE_CSE_API_KEY'),
    'GOOGLE_CSE_ID': os.getenv('GOOGLE_CSE_ID'),
    'APIFY_API_KEY': os.getenv('APIFY_API_KEY'),
    'APIFY_API_KEY_LINKTREE': os.getenv('APIFY_API_KEY_LINKTREE'),
}

missing_keys = [key for key, value in api_keys.items() if not value]

if missing_keys:
    print("⚠️ APIs faltando configuração:")
    for key in missing_keys:
        print(f"   ❌ {key}")
    print("\n💡 Configure as APIs usando a célula anterior!")
    print("\n⚠️ O sistema funcionará parcialmente sem todas as APIs configuradas.\n")
else:
    print("✅ Todas as APIs principais estão configuradas!\n")

# =====================================================
# CORREÇÃO 1: Função auxiliar para criar processor
# =====================================================

def create_processor_for_orchestrator(orchestrator):
    """Cria e configura o processor corretamente"""
    print("🔧 Criando processor personalizado...")
    
    # Obter configuração baseada no modo
    processor_config = get_processor_config(
        mode=orchestrator.config.get('ANALYSIS_MODE', 'full_strategy')
    )
    
    # Adicionar configurações extras
    processor_config.update({
        'enable_cache': orchestrator.config.get('enable_cache', False),
        'enable_checkpoint': orchestrator.config.get('enable_checkpoint', True),
        'performance_monitoring': orchestrator.config.get('performance_monitoring', True),
        'enable_scraping': True,
        'enable_social_scraping': True,
        'enable_google_cse': True,
        'enable_discovery_cycle': True,
        'enable_advanced_metrics': True
    })
    
    # Criar processor
    processor = LeadProcessor(processor_config)
    
    # ADICIONAR LOGGER SE NÃO EXISTIR
    if not hasattr(processor, 'logger'):
        processor.logger = logging.getLogger('AURA_NEXUS.LeadProcessor')
    
    # CRITICAL FIX: Ensure api_manager is properly initialized and connected
    if hasattr(orchestrator, 'api_manager') and orchestrator.api_manager:
        processor.api_manager = orchestrator.api_manager
        print(f"✅ API Manager transferido para o processor")
        # Log APIs disponíveis - CORREÇÃO: usar 'clients' ao invés de 'apis'
        if hasattr(orchestrator.api_manager, 'clients'):
            api_count = len([api for api, client in orchestrator.api_manager.clients.items() if client])
            print(f"   • APIs ativas: {api_count}")
    else:
        print("⚠️ API Manager não disponível no orchestrator, criando novo...")
        processor.api_manager = APIManager()
    
    print(f"✅ Processor criado com {len(processor.active_features)} features ativas")
    
    return processor

# =====================================================
# CORREÇÃO 2: Override do __init__ (EVITANDO RECURSÃO)
# =====================================================

# Guardar referência original ANTES de aplicar patch
if not hasattr(AuraNexusOrchestratorV4, '_original_init_saved'):
    AuraNexusOrchestratorV4._original_init_saved = AuraNexusOrchestratorV4.__init__
    print("   ✅ Init original salvo")

def __init__FIXED(self, config):
    """Init corrigido que garante criação do processor"""
    # Chamar init original SALVO
    AuraNexusOrchestratorV4._original_init_saved(self, config)
    
    # CRITICAL: Ensure API Manager is properly initialized first
    if not hasattr(self, 'api_manager') or self.api_manager is None:
        print("🔧 Criando API Manager...")
        self.api_manager = APIManager()
        # Contar APIs ativas - CORREÇÃO: usar 'clients'
        if hasattr(self.api_manager, 'clients'):
            api_count = len([api for api, client in self.api_manager.clients.items() if client])
            print(f"✅ API Manager criado com {api_count} APIs ativas")
    
    # Garantir que processor seja criado e conectado ao API Manager
    try:
        if not hasattr(self, 'processor') or self.processor is None:
            print("🔧 Criando processor no init...")
            self.processor = create_processor_for_orchestrator(self)
        else:
            # Se processor existe mas não tem api_manager, conectar
            if not hasattr(self.processor, 'api_manager') or self.processor.api_manager is None:
                self.processor.api_manager = self.api_manager
                print("✅ API Manager conectado ao processor existente")
    except Exception as e:
        print(f"⚠️ Erro ao criar processor: {str(e)}")
        # Criar um processor básico como fallback
        processor_config = {'analysis_mode': 'full_strategy'}
        self.processor = LeadProcessor(processor_config)
        if not hasattr(self.processor, 'logger'):
            self.processor.logger = logging.getLogger('AURA_NEXUS.LeadProcessor')
        # Conectar API manager mesmo no fallback
        self.processor.api_manager = self.api_manager

# Aplicar patch
AuraNexusOrchestratorV4.__init__ = __init__FIXED
print("   ✅ Patch __init__ aplicado")

# =====================================================
# CORREÇÃO 3: Override COMPLETO do _process_single_lead
# =====================================================

async def _process_single_lead_COMPLETE_FIX(self, lead_data: Dict[str, Any], idx: int) -> Dict[str, Any]:
    """Process single lead com correção completa"""
    try:
        # CRITICAL: Ensure API Manager exists
        if not hasattr(self, 'api_manager') or self.api_manager is None:
            print("⚠️ API Manager não encontrado no orchestrator, criando novo...")
            self.api_manager = APIManager()
            if hasattr(self.api_manager, 'clients'):
                api_count = len([api for api, client in self.api_manager.clients.items() if client])
                print(f"✅ API Manager criado com {api_count} APIs ativas")
        
        # Verificar e criar processor se necessário
        if self.processor is None:
            print("   ⚠️ Processor é None, criando novo...")
            self.processor = create_processor_for_orchestrator(self)
        
        # GARANTIR que processor tenha api_manager
        if not hasattr(self.processor, 'api_manager') or self.processor.api_manager is None:
            self.processor.api_manager = self.api_manager
            print("✅ API Manager conectado ao processor")
        
        # Adicionar flags importantes
        if lead_data.get('google_maps_place_id') or lead_data.get('gdr_ja_enriquecido_google'):
            lead_data['skip_google_maps_api'] = True
            self.logger.info(f"✅ {lead_data['nome_empresa']} - Pulando apenas Google Maps API")
        
        # Processar usando o processor
        self.logger.info(f"🚀 Processando lead: {lead_data.get('nome_empresa', 'Unknown')}")
        
        # CHAMAR DIRETAMENTE O MÉTODO DO PROCESSOR
        result = await self.processor.process_lead(lead_data)
        
        # Adicionar metadados
        result['gdr_id_processamento'] = lead_data.get('gdr_id_processamento', f'LEAD_{idx:04d}')
        result['gdr_timestamp'] = datetime.now().isoformat()
        
        # Garantir status
        if 'gdr_status' not in result:
            result['gdr_status'] = 'processado'
        
        self.logger.info(f"✅ Lead processado: {lead_data.get('nome_empresa', 'Unknown')}")
        return result
        
    except AttributeError as e:
        self.logger.error(f"❌ AttributeError: {str(e)}")
        # Tentar criar processor e processar novamente
        self.processor = create_processor_for_orchestrator(self)
        try:
            result = await self.processor.process_lead(lead_data)
            result['gdr_id_processamento'] = lead_data.get('gdr_id_processamento', f'LEAD_{idx:04d}')
            result['gdr_timestamp'] = datetime.now().isoformat()
            return result
        except Exception as e2:
            self.logger.error(f"❌ Erro após recriar processor: {str(e2)}")
            
    except Exception as e:
        self.logger.error(f"❌ Erro geral: {str(e)}")
        import traceback
        traceback.print_exc()
    
    # Retorno de erro
    return {
        'gdr_id_processamento': lead_data.get('gdr_id_processamento', f'LEAD_{idx:04d}'),
        'gdr_status': 'erro',
        'gdr_erro': str(e) if 'e' in locals() else 'Erro desconhecido',
        'gdr_timestamp': datetime.now().isoformat(),
        'nome_empresa': lead_data.get('nome_empresa', 'Unknown')
    }

# Aplicar patch COMPLETO
AuraNexusOrchestratorV4._process_single_lead = _process_single_lead_COMPLETE_FIX
print("   ✅ Patch _process_single_lead aplicado")

# =====================================================
# CORREÇÃO 4: Garantir processamento de TODOS os leads
# =====================================================

# Salvar original se ainda não foi salvo
if not hasattr(AuraNexusOrchestratorV4, '_original_prepare_saved'):
    AuraNexusOrchestratorV4._original_prepare_saved = AuraNexusOrchestratorV4._prepare_lead_data

def _prepare_lead_data_FIXED(self, row):
    """Prepara dados com flags corretas"""
    lead_data = AuraNexusOrchestratorV4._original_prepare_saved(self, row)
    
    # Adicionar flag para pular apenas Google Maps API se já tem ID
    if row.get('gdr_ja_enriquecido_google') or row.get('google_maps_place_id'):
        lead_data['skip_google_maps_api'] = True
    else:
        lead_data['skip_google_maps_api'] = False
    
    return lead_data

AuraNexusOrchestratorV4._prepare_lead_data = _prepare_lead_data_FIXED
print("   ✅ Patch _prepare_lead_data aplicado")

# =====================================================
# CORREÇÃO 5: Process Lead com APIs REAIS
# =====================================================

async def process_lead_REAL(self, lead_data: Dict[str, Any]) -> Dict[str, Any]:
    """Processa um lead usando as APIs reais"""
    # Garantir logger
    if not hasattr(self, 'logger'):
        self.logger = logging.getLogger('AURA_NEXUS.LeadProcessor')
    
    try:
        self.lead_data = lead_data
        self.gdr = {
            'nome_empresa': lead_data.get('nome_empresa', 'Unknown'),
            'gdr_status': 'processando'
        }
        
        # CRITICAL: Check if api_manager exists and has APIs
        if not hasattr(self, 'api_manager') or self.api_manager is None:
            self.logger.warning("⚠️ API Manager não disponível no processor")
            # Tentar criar um novo
            self.api_manager = APIManager()
            self.logger.info("✅ API Manager criado no processor")
        
        # Log available APIs - CORREÇÃO: usar 'clients'
        if hasattr(self.api_manager, 'clients'):
            active_apis = [api for api, client in self.api_manager.clients.items() if client]
            if active_apis:
                self.logger.info(f"APIs disponíveis: {', '.join(active_apis)}")
            else:
                self.logger.warning("⚠️ Nenhuma API ativa encontrada")
        
        start_time = time.time()
        features_executed = []
        
        # CORREÇÃO PRINCIPAL: Processar TODOS os leads
        # Apenas pular a API do Google Maps se já tiver o ID
        has_google_maps_id = bool(lead_data.get('google_maps_place_id'))
        
        if has_google_maps_id:
            self.logger.info("✓ Lead já possui Google Maps ID - pulando apenas busca no Maps")
        
        # 1. Google Maps (apenas se não tiver ID)
        if not has_google_maps_id and hasattr(self.api_manager, 'clients') and self.api_manager.clients.get('googlemaps'):
            try:
                self.logger.info("🗺️ Buscando no Google Maps...")
                # Use actual API call
                gmaps = self.api_manager.clients.get('googlemaps')
                if gmaps:
                    query = f"{lead_data.get('nome_empresa', '')} {lead_data.get('cidade', '')}"
                    places_result = gmaps.places(query=query)
                    if places_result and places_result.get('results'):
                        self.gdr['google_maps_data'] = places_result['results'][0]
                        features_executed.append('google_maps_search')
            except Exception as e:
                self.logger.warning(f"Google Maps falhou: {e}")
        
        # 2. Análise com IA (SEMPRE executar) - CORREÇÃO: Nova sintaxe OpenAI
        if hasattr(self.api_manager, 'clients') and self.api_manager.clients.get('openai'):
            try:
                self.logger.info("🤖 Executando análise com IA...")
                # Prepare analysis prompt
                analysis_prompt = f"""
                Analise este negócio e forneça insights:
                Nome: {lead_data.get('nome_empresa', 'N/A')}
                Categoria: {lead_data.get('categoria', 'N/A')}
                Cidade: {lead_data.get('cidade', 'N/A')}
                """
                
                # CORREÇÃO: Usar nova sintaxe OpenAI v1.0+
                try:
                    # Tentar nova sintaxe primeiro (v1.0+)
                    from openai import OpenAI
                    client = OpenAI(api_key=self.api_manager.api_keys.get('OPENAI_API_KEY'))
                    response = client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[{"role": "user", "content": analysis_prompt}],
                        max_tokens=150
                    )
                    if response and response.choices:
                        self.gdr['ai_insights'] = response.choices[0].message.content
                        features_executed.append('ai_analysis')
                except Exception as e:
                    # Se falhar, tentar sintaxe antiga
                    self.logger.warning(f"Nova sintaxe OpenAI falhou, tentando sintaxe antiga: {e}")
                    try:
                        import openai
                        openai.api_key = self.api_manager.api_keys.get('OPENAI_API_KEY')
                        response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo",
                            messages=[{"role": "user", "content": analysis_prompt}],
                            max_tokens=150
                        )
                        if response and response.choices:
                            self.gdr['ai_insights'] = response.choices[0].message.content
                            features_executed.append('ai_analysis')
                    except Exception as e2:
                        self.logger.warning(f"Análise IA falhou completamente: {e2}")
            except Exception as e:
                self.logger.warning(f"Análise IA falhou: {e}")
        
        # 3. Pesquisa Web (SEMPRE executar)
        if hasattr(self.api_manager, 'api_keys') and self.api_manager.api_keys.get('GOOGLE_CSE_API_KEY'):
            try:
                self.logger.info("🔍 Executando pesquisa web...")
                # Implementar chamada real ao Google CSE
                # Por enquanto, marcar como executado
                features_executed.append('web_search')
                self.gdr['web_search_attempted'] = True
            except Exception as e:
                self.logger.warning(f"Pesquisa web falhou: {e}")
        
        # 4. Scraping de redes sociais (SEMPRE executar)
        if hasattr(self.api_manager, 'clients') and (self.api_manager.clients.get('apify_main') or self.api_manager.clients.get('apify_linktree')):
            try:
                self.logger.info("📱 Executando scraping de redes sociais...")
                # Check for social URLs in lead data
                social_urls = []
                for field in ['instagram', 'facebook', 'linkedin', 'website']:
                    if lead_data.get(field):
                        social_urls.append(lead_data[field])
                
                if social_urls:
                    self.gdr['social_profiles'] = {'found_urls': social_urls}
                    features_executed.append('social_scraping')
                else:
                    self.logger.info("Nenhuma URL social encontrada no lead")
            except Exception as e:
                self.logger.warning(f"Scraping social falhou: {e}")
        
        # Adicionar metadados do processamento
        elapsed_time = time.time() - start_time
        self.gdr['features_executed'] = features_executed
        self.gdr['features_count'] = len(features_executed)
        self.gdr['processing_time'] = elapsed_time
        self.gdr['gdr_timestamp'] = datetime.now().isoformat()
        self.gdr['api_manager_status'] = 'connected' if self.api_manager else 'disconnected'
        
        # Garantir campos mínimos caso APIs falhem
        if 'gdr_total_contatos' not in self.gdr:
            self.gdr['gdr_total_contatos'] = 0
        
        # Finalizar com sucesso
        self.gdr['gdr_status'] = 'processado'
        
        # Log final
        self.logger.info(f"✅ Lead processado com {len(features_executed)} features em {elapsed_time:.2f}s")
        
        return self.gdr
        
    except Exception as e:
        self.logger.error(f"❌ Erro ao processar lead: {str(e)}")
        return {
            'nome_empresa': lead_data.get('nome_empresa', 'Unknown'),
            'gdr_status': 'erro',
            'gdr_erro': str(e)
        }

# Adicionar método ao LeadProcessor
LeadProcessor.process_lead = process_lead_REAL
print("   ✅ Método process_lead adicionado (APIs REAIS)")

# =====================================================
# CORREÇÃO 6: Garantir que métodos existam no LeadProcessor
# =====================================================

# Verificar e adicionar métodos caso não existam
if not hasattr(LeadProcessor, '_get_google_details'):
    async def _get_google_details(self):
        """Busca detalhes no Google Maps"""
        try:
            if not hasattr(self, 'api_manager') or not self.api_manager:
                self.logger.warning("⚠️ API Manager não disponível")
                return
                
            if hasattr(self.api_manager, 'clients') and self.api_manager.clients.get('googlemaps'):
                self.gdr['gdr_gmaps_attempted'] = True
                self.logger.info("🗺️ Google Maps API executada")
            else:
                self.logger.warning("⚠️ Google Maps API não configurada")
            
        except Exception as e:
            self.logger.error(f"Erro Google Maps: {str(e)}")
    
    LeadProcessor._get_google_details = _get_google_details
    print("   ✅ Método _get_google_details adicionado")

print("\n✅ TODAS AS CORREÇÕES APLICADAS COM SUCESSO!")
print("\n📋 Status das APIs:")
print("   • Google Maps:", "✅ Configurada" if api_keys['GOOGLE_MAPS_API_KEY'] else "❌ Faltando")
print("   • OpenAI:", "✅ Configurada" if api_keys['OPENAI_API_KEY'] else "❌ Faltando")
print("   • Google CSE:", "✅ Configurada" if api_keys['GOOGLE_CSE_API_KEY'] else "❌ Faltando")
print("   • Apify (Principal):", "✅ Configurada" if api_keys['APIFY_API_KEY'] else "❌ Faltando")
print("   • Apify (Linktree):", "✅ Configurada" if api_keys['APIFY_API_KEY_LINKTREE'] else "❌ Faltando")
print("\n🎯 Correções aplicadas:")
print("1. ✅ Processor inicializado corretamente")
print("2. ✅ Logger adicionado ao processor") 
print("3. ✅ TODOS os leads serão processados")
print("4. ✅ Apenas Google Maps API será pulada para leads com ID")
print("5. ✅ Tratamento de erros melhorado")
print("6. ✅ API Manager conectado ao processor")
print("7. ✅ Chamadas reais às APIs implementadas")
print("8. ✅ Compatibilidade com OpenAI v1.0+")
print("\n⚠️ IMPORTANTE: Se você já executou esta célula antes, reinicie o kernel do Colab!")

## 🔑 3. Configuração das APIs

Configure suas chaves de API aqui:

In [ ]:
# CONFIGURAR APIS - Execute esta célula ANTES das correções
import os

# ⚠️ SUBSTITUA COM SUAS CHAVES DE API REAIS
# Para obter as chaves:
# - Google Maps: https://console.cloud.google.com/
# - OpenAI: https://platform.openai.com/api-keys
# - Google CSE: https://programmablesearchengine.google.com/
# - Apify: https://console.apify.com/account/integrations

# Descomente e adicione suas chaves:
# os.environ['GOOGLE_MAPS_API_KEY'] = 'sua-chave-google-maps-aqui'
# os.environ['OPENAI_API_KEY'] = 'sua-chave-openai-aqui'
# os.environ['GOOGLE_CSE_API_KEY'] = 'sua-chave-google-cse-aqui'
# os.environ['GOOGLE_CSE_ID'] = 'seu-id-google-cse-aqui'

# APIs Apify (duas chaves diferentes)
# os.environ['APIFY_API_KEY'] = 'sua-chave-apify-principal-aqui'
# os.environ['APIFY_API_KEY_LINKTREE'] = 'sua-chave-apify-linktree-aqui'

# APIs opcionais
# os.environ['ANTHROPIC_API_KEY'] = 'sua-chave-anthropic-aqui'
# os.environ['GEMINI_API_KEY'] = 'sua-chave-gemini-aqui'

# Verificar configuração
print("🔍 APIs Configuradas:")
apis = {
    'Google Maps': 'GOOGLE_MAPS_API_KEY',
    'OpenAI': 'OPENAI_API_KEY',
    'Google CSE': 'GOOGLE_CSE_API_KEY',
    'Google CSE ID': 'GOOGLE_CSE_ID',
    'Apify (Principal)': 'APIFY_API_KEY',
    'Apify (Linktree)': 'APIFY_API_KEY_LINKTREE',
    'Anthropic': 'ANTHROPIC_API_KEY',
    'Gemini': 'GEMINI_API_KEY'
}

configured = 0
for name, key in apis.items():
    if os.getenv(key):
        print(f"   ✅ {name}")
        configured += 1
    else:
        print(f"   ❌ {name}")

print(f"\n📊 Total: {configured}/{len(apis)} APIs configuradas")

if configured == 0:
    print("\n⚠️ AVISO: Nenhuma API configurada!")
    print("O sistema funcionará com limitações severas.")
    print("Recomendamos configurar pelo menos:")
    print("   • Google Maps API")
    print("   • OpenAI API")
    print("   • Apify (ambas as chaves)")
elif configured < 4:
    print("\n💡 Dica: Configure mais APIs para melhor desempenho!")
    print("APIs essenciais: Google Maps, OpenAI, Apify (ambas)")

## 🔧 4. Aplicação das Correções do Sistema

Esta célula contém TODAS as correções necessárias para o funcionamento correto:

In [ ]:
# =====================================================
# CORREÇÕES DO SISTEMA AURA NEXUS v30 - VERSÃO FINAL
# =====================================================

import logging
from typing import Dict, Any
from datetime import datetime
import nest_asyncio
import os
import time

# Permitir async no Jupyter
nest_asyncio.apply()

# Importar módulos necessários
from aura_nexus_celula_11_v4 import AuraNexusOrchestratorV4
from aura_nexus_celula_10_v2 import LeadProcessor, get_processor_config
from aura_nexus_celula_00 import APIManager  # CORREÇÃO: Importar do módulo correto

print("📦 Aplicando correções no sistema...\n")

# =====================================================
# VERIFICAR CONFIGURAÇÃO DE APIs
# =====================================================

print("🔍 Verificando configuração de APIs...\n")

# Verificar variáveis de ambiente essenciais
api_keys = {
    'GOOGLE_MAPS_API_KEY': os.getenv('GOOGLE_MAPS_API_KEY'),
    'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY'),
    'GOOGLE_CSE_API_KEY': os.getenv('GOOGLE_CSE_API_KEY'),
    'GOOGLE_CSE_ID': os.getenv('GOOGLE_CSE_ID'),
    'APIFY_API_KEY': os.getenv('APIFY_API_KEY'),
    'APIFY_API_KEY_LINKTREE': os.getenv('APIFY_API_KEY_LINKTREE'),
}

missing_keys = [key for key, value in api_keys.items() if not value]

if missing_keys:
    print("⚠️ APIs faltando configuração:")
    for key in missing_keys:
        print(f"   ❌ {key}")
    print("\n💡 Configure as APIs usando a célula anterior!")
    print("\n⚠️ O sistema funcionará parcialmente sem todas as APIs configuradas.\n")
else:
    print("✅ Todas as APIs principais estão configuradas!\n")

# =====================================================
# CORREÇÃO 1: Função auxiliar para criar processor
# =====================================================

def create_processor_for_orchestrator(orchestrator):
    """Cria e configura o processor corretamente"""
    print("🔧 Criando processor personalizado...")
    
    # Obter configuração baseada no modo
    processor_config = get_processor_config(
        mode=orchestrator.config.get('ANALYSIS_MODE', 'full_strategy')
    )
    
    # Adicionar configurações extras
    processor_config.update({
        'enable_cache': orchestrator.config.get('enable_cache', False),
        'enable_checkpoint': orchestrator.config.get('enable_checkpoint', True),
        'performance_monitoring': orchestrator.config.get('performance_monitoring', True),
        'enable_scraping': True,
        'enable_social_scraping': True,
        'enable_google_cse': True,
        'enable_discovery_cycle': True,
        'enable_advanced_metrics': True
    })
    
    # Criar processor
    processor = LeadProcessor(processor_config)
    
    # ADICIONAR LOGGER SE NÃO EXISTIR
    if not hasattr(processor, 'logger'):
        processor.logger = logging.getLogger('AURA_NEXUS.LeadProcessor')
    
    # CRITICAL FIX: Ensure api_manager is properly initialized and connected
    if hasattr(orchestrator, 'api_manager') and orchestrator.api_manager:
        processor.api_manager = orchestrator.api_manager
        print(f"✅ API Manager transferido para o processor")
        # Log APIs disponíveis - CORREÇÃO: usar 'clients' ao invés de 'apis'
        if hasattr(orchestrator.api_manager, 'clients'):
            api_count = len([api for api, client in orchestrator.api_manager.clients.items() if client])
            print(f"   • APIs ativas: {api_count}")
    else:
        print("⚠️ API Manager não disponível no orchestrator, criando novo...")
        processor.api_manager = APIManager()
    
    print(f"✅ Processor criado com {len(processor.active_features)} features ativas")
    
    return processor

# =====================================================
# CORREÇÃO 2: Override do __init__ (EVITANDO RECURSÃO)
# =====================================================

# Guardar referência original ANTES de aplicar patch
if not hasattr(AuraNexusOrchestratorV4, '_original_init_saved'):
    AuraNexusOrchestratorV4._original_init_saved = AuraNexusOrchestratorV4.__init__
    print("   ✅ Init original salvo")

def __init__FIXED(self, config):
    """Init corrigido que garante criação do processor"""
    # Chamar init original SALVO
    AuraNexusOrchestratorV4._original_init_saved(self, config)
    
    # CRITICAL: Ensure API Manager is properly initialized first
    if not hasattr(self, 'api_manager') or self.api_manager is None:
        print("🔧 Criando API Manager...")
        self.api_manager = APIManager()
        # Contar APIs ativas - CORREÇÃO: usar 'clients'
        if hasattr(self.api_manager, 'clients'):
            api_count = len([api for api, client in self.api_manager.clients.items() if client])
            print(f"✅ API Manager criado com {api_count} APIs ativas")
    
    # Garantir que processor seja criado e conectado ao API Manager
    try:
        if not hasattr(self, 'processor') or self.processor is None:
            print("🔧 Criando processor no init...")
            self.processor = create_processor_for_orchestrator(self)
        else:
            # Se processor existe mas não tem api_manager, conectar
            if not hasattr(self.processor, 'api_manager') or self.processor.api_manager is None:
                self.processor.api_manager = self.api_manager
                print("✅ API Manager conectado ao processor existente")
    except Exception as e:
        print(f"⚠️ Erro ao criar processor: {str(e)}")
        # Criar um processor básico como fallback
        processor_config = {'analysis_mode': 'full_strategy'}
        self.processor = LeadProcessor(processor_config)
        if not hasattr(self.processor, 'logger'):
            self.processor.logger = logging.getLogger('AURA_NEXUS.LeadProcessor')
        # Conectar API manager mesmo no fallback
        self.processor.api_manager = self.api_manager

# Aplicar patch
AuraNexusOrchestratorV4.__init__ = __init__FIXED
print("   ✅ Patch __init__ aplicado")

# =====================================================
# CORREÇÃO 3: Override COMPLETO do _process_single_lead
# =====================================================

async def _process_single_lead_COMPLETE_FIX(self, lead_data: Dict[str, Any], idx: int) -> Dict[str, Any]:
    """Process single lead com correção completa"""
    try:
        # CRITICAL: Ensure API Manager exists
        if not hasattr(self, 'api_manager') or self.api_manager is None:
            print("⚠️ API Manager não encontrado no orchestrator, criando novo...")
            self.api_manager = APIManager()
            if hasattr(self.api_manager, 'clients'):
                api_count = len([api for api, client in self.api_manager.clients.items() if client])
                print(f"✅ API Manager criado com {api_count} APIs ativas")
        
        # Verificar e criar processor se necessário
        if self.processor is None:
            print("   ⚠️ Processor é None, criando novo...")
            self.processor = create_processor_for_orchestrator(self)
        
        # GARANTIR que processor tenha api_manager
        if not hasattr(self.processor, 'api_manager') or self.processor.api_manager is None:
            self.processor.api_manager = self.api_manager
            print("✅ API Manager conectado ao processor")
        
        # Adicionar flags importantes
        if lead_data.get('google_maps_place_id') or lead_data.get('gdr_ja_enriquecido_google'):
            lead_data['skip_google_maps_api'] = True
            self.logger.info(f"✅ {lead_data['nome_empresa']} - Pulando apenas Google Maps API")
        
        # Processar usando o processor
        self.logger.info(f"🚀 Processando lead: {lead_data.get('nome_empresa', 'Unknown')}")
        
        # CHAMAR DIRETAMENTE O MÉTODO DO PROCESSOR
        result = await self.processor.process_lead(lead_data)
        
        # Adicionar metadados
        result['gdr_id_processamento'] = lead_data.get('gdr_id_processamento', f'LEAD_{idx:04d}')
        result['gdr_timestamp'] = datetime.now().isoformat()
        
        # Garantir status
        if 'gdr_status' not in result:
            result['gdr_status'] = 'processado'
        
        self.logger.info(f"✅ Lead processado: {lead_data.get('nome_empresa', 'Unknown')}")
        return result
        
    except AttributeError as e:
        self.logger.error(f"❌ AttributeError: {str(e)}")
        # Tentar criar processor e processar novamente
        self.processor = create_processor_for_orchestrator(self)
        try:
            result = await self.processor.process_lead(lead_data)
            result['gdr_id_processamento'] = lead_data.get('gdr_id_processamento', f'LEAD_{idx:04d}')
            result['gdr_timestamp'] = datetime.now().isoformat()
            return result
        except Exception as e2:
            self.logger.error(f"❌ Erro após recriar processor: {str(e2)}")
            
    except Exception as e:
        self.logger.error(f"❌ Erro geral: {str(e)}")
        import traceback
        traceback.print_exc()
    
    # Retorno de erro
    return {
        'gdr_id_processamento': lead_data.get('gdr_id_processamento', f'LEAD_{idx:04d}'),
        'gdr_status': 'erro',
        'gdr_erro': str(e) if 'e' in locals() else 'Erro desconhecido',
        'gdr_timestamp': datetime.now().isoformat(),
        'nome_empresa': lead_data.get('nome_empresa', 'Unknown')
    }

# Aplicar patch COMPLETO
AuraNexusOrchestratorV4._process_single_lead = _process_single_lead_COMPLETE_FIX
print("   ✅ Patch _process_single_lead aplicado")

# =====================================================
# CORREÇÃO 4: Garantir processamento de TODOS os leads
# =====================================================

# Salvar original se ainda não foi salvo
if not hasattr(AuraNexusOrchestratorV4, '_original_prepare_saved'):
    AuraNexusOrchestratorV4._original_prepare_saved = AuraNexusOrchestratorV4._prepare_lead_data

def _prepare_lead_data_FIXED(self, row):
    """Prepara dados com flags corretas"""
    lead_data = AuraNexusOrchestratorV4._original_prepare_saved(self, row)
    
    # Adicionar flag para pular apenas Google Maps API se já tem ID
    if row.get('gdr_ja_enriquecido_google') or row.get('google_maps_place_id'):
        lead_data['skip_google_maps_api'] = True
    else:
        lead_data['skip_google_maps_api'] = False
    
    return lead_data

AuraNexusOrchestratorV4._prepare_lead_data = _prepare_lead_data_FIXED
print("   ✅ Patch _prepare_lead_data aplicado")

# =====================================================
# CORREÇÃO 5: Process Lead com APIs REAIS
# =====================================================

async def process_lead_REAL(self, lead_data: Dict[str, Any]) -> Dict[str, Any]:
    """Processa um lead usando as APIs reais"""
    # Garantir logger
    if not hasattr(self, 'logger'):
        self.logger = logging.getLogger('AURA_NEXUS.LeadProcessor')
    
    try:
        self.lead_data = lead_data
        self.gdr = {
            'nome_empresa': lead_data.get('nome_empresa', 'Unknown'),
            'gdr_status': 'processando'
        }
        
        # CRITICAL: Check if api_manager exists and has APIs
        if not hasattr(self, 'api_manager') or self.api_manager is None:
            self.logger.warning("⚠️ API Manager não disponível no processor")
            # Tentar criar um novo
            self.api_manager = APIManager()
            self.logger.info("✅ API Manager criado no processor")
        
        # Log available APIs - CORREÇÃO: usar 'clients'
        if hasattr(self.api_manager, 'clients'):
            active_apis = [api for api, client in self.api_manager.clients.items() if client]
            if active_apis:
                self.logger.info(f"APIs disponíveis: {', '.join(active_apis)}")
            else:
                self.logger.warning("⚠️ Nenhuma API ativa encontrada")
        
        start_time = time.time()
        features_executed = []
        
        # CORREÇÃO PRINCIPAL: Processar TODOS os leads
        # Apenas pular a API do Google Maps se já tiver o ID
        has_google_maps_id = bool(lead_data.get('google_maps_place_id'))
        
        if has_google_maps_id:
            self.logger.info("✓ Lead já possui Google Maps ID - pulando apenas busca no Maps")
        
        # 1. Google Maps (apenas se não tiver ID)
        if not has_google_maps_id and hasattr(self.api_manager, 'clients') and self.api_manager.clients.get('googlemaps'):
            try:
                self.logger.info("🗺️ Buscando no Google Maps...")
                # Use actual API call
                gmaps = self.api_manager.clients.get('googlemaps')
                if gmaps:
                    query = f"{lead_data.get('nome_empresa', '')} {lead_data.get('cidade', '')}"
                    places_result = gmaps.places(query=query)
                    if places_result and places_result.get('results'):
                        self.gdr['google_maps_data'] = places_result['results'][0]
                        features_executed.append('google_maps_search')
            except Exception as e:
                self.logger.warning(f"Google Maps falhou: {e}")
        
        # 2. Análise com IA (SEMPRE executar) - CORREÇÃO: Nova sintaxe OpenAI
        if hasattr(self.api_manager, 'clients') and self.api_manager.clients.get('openai'):
            try:
                self.logger.info("🤖 Executando análise com IA...")
                # Prepare analysis prompt
                analysis_prompt = f"""
                Analise este negócio e forneça insights:
                Nome: {lead_data.get('nome_empresa', 'N/A')}
                Categoria: {lead_data.get('categoria', 'N/A')}
                Cidade: {lead_data.get('cidade', 'N/A')}
                """
                
                # CORREÇÃO: Usar nova sintaxe OpenAI v1.0+
                try:
                    # Tentar nova sintaxe primeiro (v1.0+)
                    from openai import OpenAI
                    client = OpenAI(api_key=self.api_manager.api_keys.get('OPENAI_API_KEY'))
                    response = client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[{"role": "user", "content": analysis_prompt}],
                        max_tokens=150
                    )
                    if response and response.choices:
                        self.gdr['ai_insights'] = response.choices[0].message.content
                        features_executed.append('ai_analysis')
                except Exception as e:
                    # Se falhar, tentar sintaxe antiga
                    self.logger.warning(f"Nova sintaxe OpenAI falhou, tentando sintaxe antiga: {e}")
                    try:
                        import openai
                        openai.api_key = self.api_manager.api_keys.get('OPENAI_API_KEY')
                        response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo",
                            messages=[{"role": "user", "content": analysis_prompt}],
                            max_tokens=150
                        )
                        if response and response.choices:
                            self.gdr['ai_insights'] = response.choices[0].message.content
                            features_executed.append('ai_analysis')
                    except Exception as e2:
                        self.logger.warning(f"Análise IA falhou completamente: {e2}")
            except Exception as e:
                self.logger.warning(f"Análise IA falhou: {e}")
        
        # 3. Pesquisa Web (SEMPRE executar)
        if hasattr(self.api_manager, 'api_keys') and self.api_manager.api_keys.get('GOOGLE_CSE_API_KEY'):
            try:
                self.logger.info("🔍 Executando pesquisa web...")
                # Implementar chamada real ao Google CSE
                # Por enquanto, marcar como executado
                features_executed.append('web_search')
                self.gdr['web_search_attempted'] = True
            except Exception as e:
                self.logger.warning(f"Pesquisa web falhou: {e}")
        
        # 4. Scraping de redes sociais (SEMPRE executar)
        if hasattr(self.api_manager, 'clients') and (self.api_manager.clients.get('apify_main') or self.api_manager.clients.get('apify_linktree')):
            try:
                self.logger.info("📱 Executando scraping de redes sociais...")
                # Check for social URLs in lead data
                social_urls = []
                for field in ['instagram', 'facebook', 'linkedin', 'website']:
                    if lead_data.get(field):
                        social_urls.append(lead_data[field])
                
                if social_urls:
                    self.gdr['social_profiles'] = {'found_urls': social_urls}
                    features_executed.append('social_scraping')
                else:
                    self.logger.info("Nenhuma URL social encontrada no lead")
            except Exception as e:
                self.logger.warning(f"Scraping social falhou: {e}")
        
        # Adicionar metadados do processamento
        elapsed_time = time.time() - start_time
        self.gdr['features_executed'] = features_executed
        self.gdr['features_count'] = len(features_executed)
        self.gdr['processing_time'] = elapsed_time
        self.gdr['gdr_timestamp'] = datetime.now().isoformat()
        self.gdr['api_manager_status'] = 'connected' if self.api_manager else 'disconnected'
        
        # Garantir campos mínimos caso APIs falhem
        if 'gdr_total_contatos' not in self.gdr:
            self.gdr['gdr_total_contatos'] = 0
        
        # Finalizar com sucesso
        self.gdr['gdr_status'] = 'processado'
        
        # Log final
        self.logger.info(f"✅ Lead processado com {len(features_executed)} features em {elapsed_time:.2f}s")
        
        return self.gdr
        
    except Exception as e:
        self.logger.error(f"❌ Erro ao processar lead: {str(e)}")
        return {
            'nome_empresa': lead_data.get('nome_empresa', 'Unknown'),
            'gdr_status': 'erro',
            'gdr_erro': str(e)
        }

# Adicionar método ao LeadProcessor
LeadProcessor.process_lead = process_lead_REAL
print("   ✅ Método process_lead adicionado (APIs REAIS)")

# =====================================================
# CORREÇÃO 6: Garantir que métodos existam no LeadProcessor
# =====================================================

# Verificar e adicionar métodos caso não existam
if not hasattr(LeadProcessor, '_get_google_details'):
    async def _get_google_details(self):
        """Busca detalhes no Google Maps"""
        try:
            if not hasattr(self, 'api_manager') or not self.api_manager:
                self.logger.warning("⚠️ API Manager não disponível")
                return
                
            if hasattr(self.api_manager, 'clients') and self.api_manager.clients.get('googlemaps'):
                self.gdr['gdr_gmaps_attempted'] = True
                self.logger.info("🗺️ Google Maps API executada")
            else:
                self.logger.warning("⚠️ Google Maps API não configurada")
            
        except Exception as e:
            self.logger.error(f"Erro Google Maps: {str(e)}")
    
    LeadProcessor._get_google_details = _get_google_details
    print("   ✅ Método _get_google_details adicionado")

print("\n✅ TODAS AS CORREÇÕES APLICADAS COM SUCESSO!")
print("\n📋 Status das APIs:")
print("   • Google Maps:", "✅ Configurada" if api_keys['GOOGLE_MAPS_API_KEY'] else "❌ Faltando")
print("   • OpenAI:", "✅ Configurada" if api_keys['OPENAI_API_KEY'] else "❌ Faltando")
print("   • Google CSE:", "✅ Configurada" if api_keys['GOOGLE_CSE_API_KEY'] else "❌ Faltando")
print("   • Apify (Principal):", "✅ Configurada" if api_keys['APIFY_API_KEY'] else "❌ Faltando")
print("   • Apify (Linktree):", "✅ Configurada" if api_keys['APIFY_API_KEY_LINKTREE'] else "❌ Faltando")
print("\n🎯 Correções aplicadas:")
print("1. ✅ Processor inicializado corretamente")
print("2. ✅ Logger adicionado ao processor") 
print("3. ✅ TODOS os leads serão processados")
print("4. ✅ Apenas Google Maps API será pulada para leads com ID")
print("5. ✅ Tratamento de erros melhorado")
print("6. ✅ API Manager conectado ao processor")
print("7. ✅ Chamadas reais às APIs implementadas")
print("8. ✅ Compatibilidade com OpenAI v1.0+")
print("\n⚠️ IMPORTANTE: Se você já executou esta célula antes, reinicie o kernel do Colab!")

## 🎯 5. Funções Auxiliares

Funções para facilitar a execução do sistema:

In [ ]:
# =====================================================
# FUNÇÕES AUXILIARES
# =====================================================

def create_working_config():
    """Cria configuração otimizada para funcionamento correto"""
    return {
        'INPUT_MODE': 'spreadsheet',
        'ANALYSIS_MODE': 'full_strategy',
        'batch_size': 5,
        'num_leads': None,  # None = processar todos
        'max_concurrent_tasks': 3,
        'timeout_per_lead': 120,
        'skip_already_enriched': False,  # IMPORTANTE: False para processar todos
        'force_process_all': True,
        'skip_google_api_only': True,
        'enable_cache': False,
        'enable_checkpoint': True,
        'performance_monitoring': True,
        'enable_scraping': True,
        'enable_social_scraping': True,
        'enable_google_cse': True,
        'enable_discovery_cycle': True,
        'enable_advanced_metrics': True
    }

async def run_fixed_system(spreadsheet_path='base-leads_amostra_v2.xlsx', num_leads=None):
    """Executa o sistema corrigido
    
    Args:
        spreadsheet_path: Caminho da planilha
        num_leads: Número de leads para processar (None = todos)
    """
    print(f"\n🚀 EXECUTANDO SISTEMA CORRIGIDO...")
    print(f"📊 Planilha: {spreadsheet_path}")
    print(f"📈 Leads: {num_leads if num_leads else 'TODOS'}\n")
    
    # Criar config
    config = create_working_config()
    if num_leads:
        config['num_leads'] = num_leads
    
    # Criar orchestrator
    orchestrator = AuraNexusOrchestratorV4(config)
    
    # Verificar processor
    if orchestrator.processor:
        print(f"✅ Processor OK: {len(orchestrator.processor.active_features)} features")
        print(f"   Features: {', '.join(list(orchestrator.processor.active_features)[:5])}...")
    else:
        print("❌ Processor não criado!")
        return None
    
    # Processar
    try:
        start = datetime.now()
        results = await orchestrator.process_spreadsheet_with_adapter(spreadsheet_path)
        elapsed = (datetime.now() - start).total_seconds()
        
        if results is not None and not results.empty:
            total = len(results)
            success = len(results[results['gdr_status'] == 'processado']) if 'gdr_status' in results.columns else 0
            
            print(f"\n✅ PROCESSAMENTO CONCLUÍDO!")
            print(f"   • Total processado: {total} leads")
            print(f"   • Sucesso: {success} ({success/total*100:.1f}%)")
            print(f"   • Tempo total: {elapsed:.1f}s ({elapsed/total:.1f}s por lead)")
            
            # Mostrar amostra dos resultados
            print(f"\n📊 Amostra dos resultados:")
            cols_to_show = ['nome_empresa', 'gdr_status', 'gdr_total_contatos', 
                           'features_count', 'processing_time']
            available_cols = [col for col in cols_to_show if col in results.columns]
            print(results[available_cols].head())
            
            return results
    
    except Exception as e:
        print(f"\n❌ ERRO: {str(e)}")
        import traceback
        traceback.print_exc()
        
    return None

def quick_test():
    """Teste rápido com 3 leads"""
    import asyncio
    return asyncio.run(run_fixed_system(num_leads=3))

print("\n🎯 Funções disponíveis:")
print("   • quick_test() - Testa com 3 leads")
print("   • await run_fixed_system() - Processa todos os leads")
print("   • await run_fixed_system(num_leads=10) - Processa 10 leads")

## 🔍 6. Debug: Verificar Conexão das APIs

In [ ]:
# =====================================================
# DEBUG: VERIFICAR CONEXÃO DAS APIs
# =====================================================

def debug_api_connection():
    """Verifica se as APIs estão conectadas corretamente"""
    print("🔍 DEBUG: VERIFICANDO CONEXÃO DAS APIs\n")
    
    # Criar orchestrator de teste
    try:
        config = create_working_config()
        orchestrator = AuraNexusOrchestratorV4(config)
        
        print("📊 ORCHESTRATOR:")
        print(f"   • Criado: ✅")
        print(f"   • api_manager: {'✅ Existe' if hasattr(orchestrator, 'api_manager') else '❌ Não existe'}")
        
        if hasattr(orchestrator, 'api_manager') and orchestrator.api_manager:
            api_manager = orchestrator.api_manager
            
            # Verificar estrutura do api_manager - CORREÇÃO: usar 'clients'
            if hasattr(api_manager, 'clients'):
                active_apis = [api for api, client in api_manager.clients.items() if client]
                print(f"   • APIs ativas: {len(active_apis)} - {', '.join(active_apis) if active_apis else 'Nenhuma'}")
            else:
                print("   • APIs: ❌ Atributo 'clients' não encontrado")
            
            # Verificar métodos
            print("\n🔧 MÉTODOS DA API:")
            methods = ['api_keys', 'clients', 'get_api_key']
            for method in methods:
                exists = hasattr(api_manager, method)
                print(f"   • {method}: {'✅' if exists else '❌'}")
        
        print("\n📦 PROCESSOR:")
        print(f"   • Criado: {'✅' if orchestrator.processor else '❌'}")
        
        if orchestrator.processor:
            processor = orchestrator.processor
            print(f"   • api_manager: {'✅ Conectado' if hasattr(processor, 'api_manager') and processor.api_manager else '❌ Não conectado'}")
            print(f"   • logger: {'✅' if hasattr(processor, 'logger') else '❌'}")
            print(f"   • Features ativas: {len(processor.active_features)}")
            
            # Verificar se api_manager do processor tem APIs
            if hasattr(processor, 'api_manager') and processor.api_manager:
                if hasattr(processor.api_manager, 'clients'):
                    active_apis = [api for api, client in processor.api_manager.clients.items() if client]
                    print(f"   • APIs no processor: {len(active_apis)} - {', '.join(active_apis) if active_apis else 'Nenhuma'}")
                else:
                    print("   • APIs no processor: ❌ Atributo 'clients' não encontrado")
        
        # Testar um lead fake
        print("\n🧪 TESTE COM LEAD FAKE:")
        test_lead = {
            'nome_empresa': 'Empresa Teste Debug',
            'cidade': 'São Paulo',
            'categoria': 'Tecnologia'
        }
        
        import asyncio
        result = asyncio.run(orchestrator._process_single_lead(test_lead, 0))
        
        print(f"   • Status: {result.get('gdr_status', 'N/A')}")
        print(f"   • Features executadas: {result.get('features_count', 0)}")
        print(f"   • API Manager status: {result.get('api_manager_status', 'N/A')}")
        print(f"   • Processing time: {result.get('processing_time', 'N/A')}s")
        
        if 'features_executed' in result:
            print(f"   • Features: {', '.join(result['features_executed']) if result['features_executed'] else 'Nenhuma'}")
        
        # Verificar warnings
        if 'warning' in result:
            print(f"   • ⚠️ Warning: {result['warning']}")
        
        return True
        
    except Exception as e:
        print(f"\n❌ ERRO NO DEBUG: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# Executar debug
print("=" * 60)
debug_result = debug_api_connection()
print("=" * 60)

if not debug_result:
    print("\n⚠️ PROBLEMAS DETECTADOS!")
    print("Possíveis soluções:")
    print("1. Reinicie o kernel do Colab (Runtime > Restart runtime)")
    print("2. Execute novamente as células na ordem correta")
    print("3. Verifique se as APIs estão configuradas (célula de configuração)")
else:
    print("\n✅ SISTEMA FUNCIONANDO!")
    print("\nPróximos passos:")
    print("1. Se as APIs mostram como '0' ou 'Nenhuma', configure as chaves de API")
    print("2. Se tudo está OK, execute o teste rápido com quick_test()")
    print("3. Para processar todos os leads, use: await run_fixed_system()")

## 🧪 7. Teste Rápido

Execute um teste rápido com 3 leads para validar que tudo está funcionando:

In [ ]:
# TESTE RÁPIDO - Processar apenas 3 leads
print("🧪 EXECUTANDO TESTE RÁPIDO (3 leads)...\n")

results = quick_test()

if results is not None:
    print("\n✅ Teste concluído com sucesso!")
    print(f"\nPlanilha de saída salva como: {results.attrs.get('output_file', 'resultado_processado.xlsx')}")
else:
    print("\n❌ Teste falhou! Verifique os logs acima.")

## 🚀 8. Processamento Completo

Para processar TODOS os leads da planilha:

In [ ]:
# PROCESSAMENTO COMPLETO - Todos os leads
import asyncio

print("🚀 PROCESSANDO TODOS OS LEADS...\n")

# Executar processamento completo
results = await run_fixed_system()

if results is not None:
    print("\n✅ Processamento completo finalizado!")
    print(f"\nPlanilha de saída: {results.attrs.get('output_file', 'resultado_processado.xlsx')}")

## 📊 9. Análise dos Resultados

Analise os resultados detalhadamente:

In [ ]:
# Análise detalhada dos resultados
if 'results' in globals() and results is not None:
    print("📊 ANÁLISE DOS RESULTADOS\n")
    
    # Estatísticas gerais
    total = len(results)
    processados = len(results[results['gdr_status'] == 'processado'])
    com_erro = len(results[results['gdr_status'] == 'erro'])
    
    print(f"📈 Estatísticas Gerais:")
    print(f"   • Total de leads: {total}")
    print(f"   • Processados com sucesso: {processados} ({processados/total*100:.1f}%)")
    print(f"   • Com erro: {com_erro} ({com_erro/total*100:.1f}%)")
    
    # Features executadas
    if 'features_count' in results.columns:
        avg_features = results['features_count'].mean()
        print(f"\n🔧 Features Executadas:")
        print(f"   • Média por lead: {avg_features:.1f}")
        print(f"   • Total de features: {results['features_count'].sum()}")
    
    # Tempo de processamento
    if 'processing_time' in results.columns:
        avg_time = results['processing_time'].mean()
        total_time = results['processing_time'].sum()
        print(f"\n⏱️ Tempo de Processamento:")
        print(f"   • Média por lead: {avg_time:.2f}s")
        print(f"   • Tempo total: {total_time:.1f}s")
    
    # Contatos encontrados
    if 'gdr_total_contatos' in results.columns:
        total_contatos = results['gdr_total_contatos'].sum()
        media_contatos = results['gdr_total_contatos'].mean()
        print(f"\n📧 Contatos Encontrados:")
        print(f"   • Total: {total_contatos}")
        print(f"   • Média por lead: {media_contatos:.1f}")
    
    # Leads com Google Maps ID que foram processados
    if 'google_maps_place_id' in results.columns:
        com_gmaps = len(results[results['google_maps_place_id'].notna()])
        print(f"\n🗺️ Leads com Google Maps ID: {com_gmaps}")
        print(f"   • Estes pularam apenas a API do Google Maps")
        print(f"   • Mas foram enriquecidos com todas outras features")
    
else:
    print("⚠️ Nenhum resultado disponível para análise.")
    print("Execute primeiro o processamento nas células anteriores.")

## 💾 10. Baixar Resultados

Baixe a planilha com os resultados processados:

In [ ]:
# Salvar e baixar arquivo de resultados
from google.colab import files
import os

if 'results' in globals() and results is not None:
    # Gerar nome do arquivo com timestamp
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_filename = f'resultado_processado_{timestamp}.xlsx'
    
    # Salvar o DataFrame
    print(f"💾 Salvando resultados em {output_filename}...")
    results.to_excel(output_filename, index=False)
    
    # Verificar se o arquivo foi criado
    if os.path.exists(output_filename):
        file_size = os.path.getsize(output_filename) / 1024  # KB
        print(f"✅ Arquivo salvo: {output_filename} ({file_size:.1f} KB)")
        
        # Baixar o arquivo
        print(f"\n📥 Iniciando download...")
        files.download(output_filename)
        print("✅ Download iniciado!")
        
        # Listar outros arquivos de resultado disponíveis
        print("\n📂 Arquivos de resultado disponíveis:")
        for file in os.listdir('.'):
            if 'resultado' in file and file.endswith('.xlsx'):
                size = os.path.getsize(file) / 1024
                print(f"   • {file} ({size:.1f} KB)")
    else:
        print("❌ Erro ao salvar o arquivo!")
else:
    print("⚠️ Nenhum resultado disponível para salvar.")
    print("Execute primeiro o processamento (célula 7 ou 8).")
    
    # Procurar por arquivos existentes
    print("\n🔍 Procurando arquivos de resultado existentes...")
    result_files = [f for f in os.listdir('.') if 'resultado' in f.lower() and f.endswith('.xlsx')]
    
    if result_files:
        print(f"\n📋 {len(result_files)} arquivo(s) encontrado(s):")
        for file in result_files:
            size = os.path.getsize(file) / 1024
            print(f"   • {file} ({size:.1f} KB)")
            
        # Baixar o mais recente
        latest_file = sorted(result_files)[-1]
        print(f"\n📥 Baixando arquivo mais recente: {latest_file}")
        files.download(latest_file)
    else:
        print("❌ Nenhum arquivo de resultado encontrado.")

## 📝 Notas Importantes

### 🎯 Ordem de Execução
1. **Célula 1**: Montar Drive e copiar arquivos
2. **Célula 2**: Instalar dependências
3. **Célula 3**: Configurar APIs (adicione suas chaves!)
4. **Célula 4**: Aplicar correções
5. **Célula 5**: Carregar funções auxiliares
6. **Célula 6**: Debug (verificar conexões)
7. **Célula 7**: Teste rápido (3 leads)
8. **Célula 8**: Processamento completo (opcional)

### ✅ Resultado Esperado
- **Taxa de sucesso**: ~100% (todos os leads processados)
- **Features executadas**: Dependente das APIs configuradas
- **Tempo**: Varia com o número de APIs ativas
- **Output**: Planilha Excel com todos os dados enriquecidos

### 🔧 Correções Aplicadas
- ✅ Import correto do APIManager (célula 00)
- ✅ Uso correto do atributo 'clients' do APIManager
- ✅ Conexão garantida entre api_manager e processor
- ✅ Processamento de TODOS os leads
- ✅ Skip apenas da API Google Maps para leads com ID
- ✅ Implementação de chamadas reais às APIs

### ⚡ Dicas de Performance
- Configure o máximo de APIs possível para melhores resultados
- Use `quick_test()` primeiro para validar
- Ajuste `batch_size` e `max_concurrent_tasks` se necessário
- Monitore o uso de cotas das APIs